In [112]:
import pandas as pd
import numpy as np

In [113]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [114]:
train['Family']=train['Parch']+train['SibSp']+1
test['Family']=test['Parch']+test['SibSp']+1

In [115]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [116]:
train['Family'].value_counts()

Family
1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: count, dtype: int64

In [117]:
pass_id_test=test['PassengerId'].values

In [118]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Family'],
      dtype='object')

In [119]:
def family_size_calculate(num):
    if num == 1:
        return "Alone"
    elif num>1 and num<5:
        return 'Medium'
    else:
        return 'Large'

train['Family_Size']=train['Family'].apply(family_size_calculate)
test['Family_Size']=test['Family'].apply(family_size_calculate)

In [120]:
train.drop(columns=['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Family'],inplace=True)
test.drop(columns=['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Family'],inplace=True)

In [121]:
tr_pclass_1=train[train['Pclass']==1]
tr_pclass_2=train[train['Pclass']==2]
tr_pclass_3=train[train['Pclass']==3]

In [122]:
train_mean = train['Age'].mean()
train_std = train['Age'].std()

lower_range_train = train_mean - train_std
higher_range_train = train_mean + train_std

# Generate random ages for missing values in train
train_age = np.random.uniform(lower_range_train, higher_range_train, train['Age'].isnull().sum())

# Filling NaN values by standard deviation and mean in test
test_mean = test['Age'].mean()
test_std = test['Age'].std()

lower_range_test = test_mean - test_std
higher_range_test = test_mean + test_std

# Generate random ages for missing values in test
test_age = np.random.uniform(lower_range_test, higher_range_test, test['Age'].isnull().sum())

# Replacing the values in train
train.loc[train['Age'].isnull(), 'Age'] = train_age

# Replacing the values in test
test.loc[test['Age'].isnull(), 'Age'] = test_age

In [123]:
train['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [124]:
train['Embarked']=train['Embarked'].fillna('S')

In [125]:
test['Embarked']=test['Embarked'].fillna('S')

In [126]:
test['Fare']=test['Fare'].fillna(32)

In [127]:
train = pd.get_dummies(train,columns=['Pclass','Sex','Embarked','Family_Size'],drop_first=True)
test = pd.get_dummies(test,columns=['Pclass','Sex','Embarked','Family_Size'],drop_first=True)

In [128]:
train.shape

(891, 10)

In [129]:
X=train.iloc[:,1:].values
y=train.iloc[:,0]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [130]:
X.shape

(891, 9)

In [131]:
# import optuna
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.ensemble import RandomForestClassifier

# def objective(trial):
#     # Define the hyperparameters and their possible values
#     n_estimators = trial.suggest_int('n_estimators', 50, 400)
#     max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
#     max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

#     # Create the RandomForestClassifier with the suggested hyperparameters
#     clf = RandomForestClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features,
#         random_state=42
#     )

#     # Use cross-validation to evaluate the model
#     score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3)
#     accuracy = score.mean()
#     return accuracy

# # Create a study and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best hyperparameters
# best_params = study.best_params
# print("Best hyperparameters: ", best_params)

# # Train and evaluate the model with the best hyperparameters
# best_clf = RandomForestClassifier(**best_params, random_state=42)
# best_clf.fit(X_train, y_train)
# accuracy = best_clf.score(X_test, y_test)
# print("Test accuracy: ", accuracy)

In [132]:

import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
        n_estimators=315,
        max_depth=20,
        min_samples_split=13,
        min_samples_leaf=1,
        max_features= 'log2',
        random_state=42
    )

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [133]:
accuracy = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()
accuracy*100

83.00417213298822